In [1]:
from google.colab import drive
import os
if not os.path.exists('/gd'):
    drive.mount('/gd')

Mounted at /gd


In [2]:

import locale
def gpe(x=None):
    return "UTF-8"
locale.getpreferredencoding = gpe

In [3]:
!pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu unbabel-comet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
languages = [
    ("it", "ita_Latn"),
    ("fi", "fin_Latn"),
    ("es", "spa_Latn"),
    ("el", "ell_Grek"),
    ("lt", "lit_Latn"),
    ("cs", "ces_Latn"),
    ("da", "dan_Latn"),
    ("de", "deu_Latn"),
    ("et", "est_Latn"),
    ("fr", "fra_Latn"),
    ("hu", "hun_Latn"),
    ("lv", "lvs_Latn"),
    ("mt", "mlt_Latn"),
    ("nl", "nld_Latn"),
    ("pl", "pol_Latn"),
    ("pt", "por_Latn"),
    ("sk", "slk_Latn"),
    ("sl", "slv_Latn")
]

In [6]:
dataset = {}
for lang, langcode in languages + [("en", "eng_Latn")]:
  filename = f'/content/drive/MyDrive/translation_ECB/data_multi/multi.ECB.{lang}.txt'
  with open(filename, "r") as f:
    lines = f.readlines()
  dataset[f"{lang}"] = [l.strip() for l in lines][-50:]
  print(lang, len(dataset[lang]))


it 50
fi 50
es 50
el 50
lt 50
cs 50
da 50
de 50
et 50
fr 50
hu 50
lv 50
mt 50
nl 50
pl 50
pt 50
sk 50
sl 50
en 50


Loading model

In [ ]:
from transformers import NllbTokenizer
from tqdm.auto import tqdm, trange
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model_name = 'facebook/nllb-200-distilled-600M'
#model_name = "facebook/nllb-200-3.3B"

In [ ]:
tokenizer = NllbTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import gc
import random
import numpy as np
import torch
from tqdm.auto import tqdm, trange
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [ ]:
model.cuda();

Translate

In [ ]:
def translate(text, model, src_lang='eng_Latn', tgt_lang='afr_Latn', a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    #print(inputs.input_ids.shape[1], result.shape[1])
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:

results = {}
for lang, langcode in languages:
  print(lang)
  prefix = f"/content/drive/MyDrive/translation_ECB/results/nllb-600M-sample50"
  results[f"en2{lang}"] = [translate(t, model, 'eng_Latn', langcode)[0] for t in tqdm(dataset["en"])]
  with open(f"{prefix}.en2{lang}.txt", "w") as f:
    f.write("\n".join(results[f"en2{lang}"]))
  results[f"{lang}2en"] = [translate(t, model, langcode, 'eng_Latn')[0] for t in tqdm(dataset[lang])]
  with open(f"{prefix}.{lang}2en.txt", "w") as f:
    f.write("\n".join(results[f"{lang}2en"]))


lv


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

mt


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

sl


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# Evaluate

import sacrebleu
bleu_calc = sacrebleu.BLEU()
chrf2_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

from comet import download_model, load_from_checkpoint
# Choose your model from Hugging Face Hub
# model_path = download_model("Unbabel/XCOMET-XL")
# or for example:
model_path = download_model("Unbabel/wmt22-comet-da")
# Load the model checkpoint:
comet_model = load_from_checkpoint(model_path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [9]:
# Load results from files

results = {}
for lang, langcode in languages:
  print(lang)
  prefix = f"/content/drive/MyDrive/translation_ECB/results/nllb-600M-sample50"
  with open(f"{prefix}.en2{lang}.txt", "r") as f:
    results[f'en2{lang}'] = f.readlines()
  with open(f"{prefix}.{lang}2en.txt", "r") as f:
    results[f'{lang}2en'] = f.readlines()

it
fi
es
el
lt
cs
da
de
et
fr
hu
lv
mt
nl
pl
pt
sk
sl


In [13]:
scores = {}

for lang, langcode in languages:
  scores[f'{lang}2en'] = {}
  scores[f'en2{lang}'] = {}
  #print(f"en-2-{lang}")
  en2xx_bleu = bleu_calc.corpus_score(results[f'en2{lang}'], [dataset[f'{lang}']])
  en2xx_chrf = chrf2_calc.corpus_score(results[f'en2{lang}'], [dataset[f'{lang}']])
  #print(f"{lang}-2-en")
  xx2en_bleu = bleu_calc.corpus_score(results[f'{lang}2en'], [dataset[f'en']])
  xx2en_chrf = chrf2_calc.corpus_score(results[f'{lang}2en'], [dataset[f'en']])

  scores[f'en2{lang}']['bleu'] = en2xx_bleu
  scores[f'en2{lang}']['chrf'] = en2xx_chrf
  scores[f'{lang}2en']['bleu'] = xx2en_bleu
  scores[f'{lang}2en']['chrf'] = xx2en_chrf

  # COMET data format
  data_en2xx = []
  data_xx2en = []
  for src, mt, ref in zip(dataset['en'], results[f'en2{lang}'], dataset[lang]):
    data_en2xx.append({"src": src, "mt": mt, "ref": ref})
  for src, mt, ref in zip(dataset[lang], results[f'{lang}2en'], dataset['en']):
    data_xx2en.append({"src": src, "mt": mt, "ref": ref})
  # Call predict method:
  comet_en2xx_out = comet_model.predict(data_en2xx, batch_size=8)#, gpus=1)
  comet_xx2en_out = comet_model.predict(data_xx2en, batch_size=8)
  scores[f'en2{lang}']['comet'] = comet_en2xx_out.system_score
  scores[f'{lang}2en']['comet'] = comet_xx2en_out.system_score
  #print(lang, comet_en2xx_out.system_score, comet_xx2en_out.system_score)

  print(f"{lang},en2{lang},{en2xx_bleu.score},{en2xx_chrf.score},{comet_en2xx_out.system_score},{lang}2en,{xx2en_bleu.score},{xx2en_chrf.score},{comet_xx2en_out.system_score}")

for lang, _ in languages:
  print(f"{lang},en2{lang},{scores[f'en2{lang}']['bleu']},{scores[f'en2{lang}']['chrf']},{scores[f'en2{lang}']['comet']},{lang}2en,{scores[f'{lang}2en']['bleu']},{scores[f'{lang}2en']['chrf']},{scores[f'{lang}2en']['comet']}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 7/7 [01:55<00:00, 16.48s/it]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.p

it,en2it,34.70510376394811,60.84013988365222,0.8575507998466492,it2en,41.05767755101685,64.64551795754507,0.8414072036743164


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 7/7 [01:48<00:00, 15.57s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False,

fi,en2fi,15.884959765025762,45.29458889086313,0.8500157654285431,fi2en,27.589205859370303,52.546818393603566,0.8374087935686112


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 7/7 [01:57<00:00, 16.72s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False,

...

pt,en2pt,41.959399144217926,63.90359956256344,0.8530441927909851,pt2en,48.85472583749973,70.16044185634327,0.8559836149215698


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 7/7 [01:56<00:00, 16.65s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False,

sk,en2sk,25.697720019798165,50.560762474498375,0.8484578692913055,sk2en,33.703449442507484,57.749266355769166,0.8279600030183792


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 7/7 [01:51<00:00, 15.89s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.

sl,en2sl,25.280582747633886,52.006396610607155,0.8441474980115891,sl2en,34.45110920173485,59.67571860677917,0.8274221670627594
it,en2it,BLEU = 34.71 64.5/40.9/28.1/20.3 (BP = 0.991 ratio = 0.991 hyp_len = 1733 ref_len = 1748),chrF2++ = 60.84,0.8575507998466492,it2en,BLEU = 41.06 72.8/47.7/34.5/26.3 (BP = 0.974 ratio = 0.975 hyp_len = 1583 ref_len = 1624),chrF2++ = 64.65,0.8414072036743164
fi,en2fi,BLEU = 15.88 42.5/20.1/11.5/6.8 (BP = 0.988 ratio = 0.988 hyp_len = 1102 ref_len = 1115),chrF2++ = 45.29,0.8500157654285431,fi2en,BLEU = 27.59 64.8/36.9/23.7/15.4 (BP = 0.903 ratio = 0.907 hyp_len = 1473 ref_len = 1624),chrF2++ = 52.55,0.8374087935686112
es,en2es,BLEU = 45.74 72.1/51.7/39.0/30.1 (BP = 1.000 ratio = 1.001 hyp_len = 1882 ref_len = 1881),chrF2++ = 66.30,0.8494695889949798,es2en,BLEU = 45.20 75.7/54.7/41.5/31.5 (BP = 0.937 ratio = 0.939 hyp_len = 1525 ref_len = 1624),chrF2++ = 67.26,0.8474746042490006
el,en2el,BLEU = 26.59 61.0/35.6/22.5/14.5 (BP = 0.916 ratio = 0.920 hyp_len = 1